In [ ]:
# ====================================================================
# 🌌 QEPC UNIVERSAL LOADER
# ====================================================================
import sys, os
# Add the project root (one level up) to the path so we can import the context
sys.path.append(os.path.abspath('..')) 

# Load everything from the central context file
from notebook_context import * # Trigger the success message
print_ready_message()
print("="*80)

-------------------------------
----------------------------------------


# ### 🧪 1. BACKTEST CONFIGURATION
# Select a date range to test the model against.
# For this v1 backtest, we will just load the schedule and look at past games.

qa.qepc_step("Initializing Backtest Pipeline...")

# 1. Load All Data
schedule = qa.load_games()
# Filter for games that have likely happened (you can adjust this logic)
# For now, we run the whole schedule to see the spread distribution
backtest_schedule = schedule.copy() 

print(f"Loaded {len(backtest_schedule)} games for backtesting.")

# 2. Load Real Strengths
team_stats = qa.load_team_stats()

In [ ]:
# ### 🏃 2. RUN MODEL
qa.qepc_step("Calculating Lambda & Simulating...")

# Calculate expected scores
schedule_with_lambda = compute_lambda(backtest_schedule, team_stats)

# Run simulation (Lower trials for speed during backtest dev)
results = run_qepc_simulation(schedule_with_lambda, num_trials=5000)

qa.qepc_step("Backtest Simulation Complete.")

In [ ]:
# ### 📊 3. ANALYZE RESULTS (Distribution Check)
# Check if the spread distribution looks normal (e.g., mostly between -15 and +15)

qa.qepc_step("Spread Distribution Analysis")

fig = px.histogram(
    results, 
    x="Expected_Spread", 
    nbins=30,
    title="Distribution of Predicted Spreads (Should be Bell Curve centered near 0)",
    template="plotly_dark"
)
fig.show()

# Show the extreme outliers to debug specific teams
outliers = results[results['Expected_Spread'].abs() > 20]
if not outliers.empty:
    print("⚠️ Extreme Outliers Detected (Spread > 20):")
    display(outliers[['Date', 'Away Team', 'Home Team', 'Expected_Spread', 'Sim_Home_Score', 'Sim_Away_Score']])
else:
    print("✅ No extreme outliers found. Model logic looks stable.")